In [29]:
filename="/mnt/c/Users/zohai/Documents/Research/Neuro_ML/LVOs_Follow_Up_CTs/BJH_2018_2019_master_v2.xlsx"

In [30]:
import pandas as pd
raw_df=pd.read_excel(io=filename)

raw_df.head()

,Unnamed: 0,Subject.label,MRN,GENISIS.CNDA,Accession.Number,first_name,last_name,dob,age.from.dob,age.radiology,...,fu_CT2_report,fu_CT3_date_time,fu_CT3_report,number_fu_cts,DHC,Deterioration,Death,Osmotics,Comments,manual_mls
0,1,BJH_001,42235384,NaN,53710639,MELODIE,SCHALK,1955-02-25,64.711841,64,...,NaN,1899-12-30 00:00:00,NaN,1,0.0,0,0.0,0.0,0,NaN
1,2,BJH_002,2484331,NaN,53702325,SHARON,HOFFMAN,1946-05-23,73.470226,73,...,NaN,0,NaN,1,0.0,"CE, MLS",1.0,0.0,WOC,4.0
2,3,BJH_003,1052729,NaN,53682761,KATHLEEN,ERNST,1947-09-11,72.156057,72,...,There is evolution of the large left anterior ...,0,NaN,2,0.0,"CE, MLS",0.0,0.0,0,4.0
3,4,BJH_004,42914360,NaN,53681846,EDDIE,SIMS,1938-01-16,81.806982,81,...,Again seen is mild hypoattenuation within a pa...,2019-11-22 13:11:00,An evolving subacute ischemic infarction invol...,3,0.0,0,0.0,0.0,0,NaN
4,5,BJH_005,5364336,NaN,53678983,MICHAEL,SANDERS,1955-06-30,64.355921,64,...,NaN,0,NaN,0,0.0,0,0.0,0.0,0,NaN


In [31]:
reports = raw_df[["fu_CT_report", "fu_CT2_report", "fu_CT3_report"]]
reports

,fu_CT_report,fu_CT2_report,fu_CT3_report
0,There is no acute intracranial hemorrhage. Ven...,NaN,NaN
1,There is large right anterior circulation isch...,NaN,NaN
2,There is evolution of large left anterior and ...,There is evolution of the large left anterior ...,NaN
3,There is no acute intracranial hemorrhage. Pre...,Again seen is mild hypoattenuation within a pa...,An evolving subacute ischemic infarction invol...
4,NaN,NaN,NaN
...,...,...,...
117,NaN,NaN,NaN
118,NaN,NaN,NaN
119,Redemonstrated is a long segment clot within t...,Long segment clot within the left middle cereb...,NaN
120,There is moderate-severe periventricular white...,NaN,NaN


In [32]:
import networkx as nx
import pyConTextNLP.pyConText as pyConText
import pyConTextNLP.itemData as itemData

In [33]:
#pyConTextNLP uses yml files to store entities and their regex pattern information
#This function is to be used for quick lookup of an entity for short term use
#Or to add new entities to a yml file

#"/mnt/c/Users/zohai/Documents/Research/Neuro_ML/Sample_Data/modifiers.yml"
#"/mnt/c/Users/zohai/Documents/Research/Neuro_ML/Sample_Data/targets.yml"

class pyConText_itemData_initializer:
    
    def __init__(self, modifiers_path=None, targets_path=None):
        if modifiers_path is None:
            self.modifiers_path = "https://raw.githubusercontent.com/chapmanbe/pyConTextNLP/master/KB/lexical_kb_05042016.yml"
        else:
            self.modifiers_path = modifiers_path
            
        self.modifiers = itemData.get_items(self.modifiers_path)
        self.targets = []
            
        if targets_path is not None:
            self.targets_path = targets_path
            self.targets = itemData.get_items(self.targets_path)
    
    def add_target(self,entity,category='', regex='', direction=''):
        self.targets.append(itemData.contextItem([entity,category,regex,direction]))
        
        #Code from pyConTextNLP itemData.py
        #def get_items(_file): context_items =  [contextItem((d["Lex"], d["Type"],r"%s"%d["Regex"],d["Direction"])) for d in yaml.load_all(f0)]
    
    def add_entity_to_yml(self, yml_path, entity, category="''", regex="''", direction="''"):
        yml_entity = "Direction: "+ direction + "\n" + \
        "Lex: "+ entity + "\n" + \
        "Regex: "+ regex + "\n" + \
        "Type: " + category + "\n" + \
        "---\n" 
        
        with open(yml_path, "r+") as f:
            old = f.read()
            f.seek(0)
            #Prepending
            f.write(yml_entity + old)
            
        f.close()
        
    def get_itemData(self):
        return self.modifiers,self.targets
        


In [34]:
#Function to determine existence or rule out given entity
#Ex. Midline shift/No midline shift
#Ex. Occlusion/No occlusion

#Input is string entity that you want to rule in/out
#***Since words have DIFFERENT forms, consider using a PREFIX INSTEAD
#Ex. Entity: occlusion; sentence contains 'occluded thrombus'
# Using the prefix 'occlu' would capture both in a rudimentary fashion

class modifier_target_finder:
    def __init__(self, sentence, mod_itemData, targ_itemData):
        self.modifiers = mod_itemData
        self.targets = targ_itemData
        self.sentence = sentence
        
        self.markup = pyConText.ConTextMarkup()
        self.markup.setRawText(sentence.lower())
        self.markup.cleanText()
        
        self.markup.markItems(self.modifiers,mode="modifier")
        self.markup.markItems(self.targets,mode="target")
        self.markup.applyModifiers()
    
    #New addition
    def log(self):
        print(self.markup.__str__())
        
    
    def get_markup(self):
        return list(self.markup.edges())
            
    def find_mod_target_pair(self,desired_modifier,modifier_attrib,desired_target,target_attrib):
        #set removes duplicates
        return list(set([(modifier.getLiteral(),target.getLiteral()) 
                for (modifier,target) in self.markup.edges() 
                # getattr() will run modifier/target . getLiteral()/categoryString()/getTagID()/getConTextCategory()
                if (desired_modifier in getattr(modifier,modifier_attrib)() 
                    and desired_target in getattr(target,target_attrib)())]))
    
    
    #Returns 0 if negated
    #Returns 1 if present and not negated
    #Returns 2 if confirmed existence
    #Returns None if not in sentence
    def confirm_existence(self, entity, entity_type):
        entities = []
        for modifier,target in self.markup.edges():
            etype_func = {"getLiteral":[modifier.getLiteral,target.getLiteral],
                          "categoryString":[modifier.categoryString,target.categoryString]}
            for func in etype_func[entity_type]:
                entities.append(func())
        if entity in entities:
            neg_pairs_list = self.find_mod_target_pair('negated_existence','categoryString',entity,entity_type)
            if neg_pairs_list:
                return 0
            def_exis_pairs_list = self.find_mod_target_pair(('definite_existence'),
                                                    'categoryString', entity,entity_type)
            prob_exis_pairs_list = self.find_mod_target_pair(('definite_existence'),
                                                            'categoryString', entity,entity_type)
            if def_exis_pairs_list or prob_exis_pairs_list:
                return 2
            return 1
        else:
            return None

In [35]:
mod_path = "/mnt/c/Users/zohai/Documents/Research/Neuro_ML/LVOs_Follow_Up_CTs/modifiers_fu.yml"
targ_path = "/mnt/c/Users/zohai/Documents/Research/Neuro_ML/LVOs_Follow_Up_CTs/targets_fu.yml"
itemData_init = pyConText_itemData_initializer(mod_path,targ_path)
itemData_init.modifiers[3]

/mnt/c/Users/zohai/Documents/Research/Neuro_ML/miniconda3/envs/neuroml/lib/python3.8/site-packages/pyConTextNLP/itemData.py:40: YAMLLoadWarning: calling yaml.load_all() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  context_items =  [contextItem((d["Lex"],


literal<<\br\b>>; category<<['right_side']>>; re<<\br\b>>; rule<<forward>>

In [36]:
mtf = modifier_target_finder("There is evolution of large left anterior and middle cerebral artery territory infarcts with loss of gray-white matter differentiation and loss of the sulci",
                             itemData_init.get_itemData()[0],itemData_init.get_itemData()[1])
mtf.get_markup()

[(<id> 165709866153094661810274258633277371541 </id> <phrase> sulc </phrase> <category> ['sulci'] </category> ,
  <id> 165712001352074421234172404642738926741 </id> <phrase> territory </phrase> <category> ['brain_anatomy'] </category> ),
 (<id> 165709866153094661810274258633277371541 </id> <phrase> sulc </phrase> <category> ['sulci'] </category> ,
  <id> 165711766044431753869089751817206428821 </id> <phrase> loss </phrase> <category> ['effacement'] </category> ),
 (<id> 165709866153094661810274258633277371541 </id> <phrase> sulc </phrase> <category> ['sulci'] </category> ,
  <id> 165712112271501941204245035604269397141 </id> <phrase> white matter </phrase> <category> ['brain_anatomy'] </category> ),
 (<id> 165709866153094661810274258633277371541 </id> <phrase> sulc </phrase> <category> ['sulci'] </category> ,
  <id> 165711808035357886429188676395500106901 </id> <phrase> loss </phrase> <category> ['effacement'] </category> ),
 (<id> 165709866153094661810274258633277371541 </id> <phrase>

In [37]:
mtf.confirm_existence("effacement","categoryString")

2

In [38]:
#if mtf.find_mod_target_pair("effacement","getLiteral","sulc","getLiteral"):
if mtf.find_mod_target_pair("efface","getLiteral","sulc","getLiteral") and mtf.confirm_existence("effacement","categoryString"):
    print(True)

In [39]:
reports["fu_CT_report"][0].split('.')

['There is no acute intracranial hemorrhage',
 ' Ventricles are of normal size and morphology',
 ' No mass effect or midline shift is present',
 ' The gray-white matter differentiation is normal',
 ' There is mild scattered periventricular white matter hypoattenuation that is nonspecific and likely represents chronic ischemic small vessel disease',
 ' ']

In [40]:
import re
class edema_extractor:
    def __init__(self, reports):
        self.reports=reports
        
        #Setting up pyConText
        self.mod_path = "/mnt/c/Users/zohai/Documents/Research/Neuro_ML/LVOs_Follow_Up_CTs/modifiers_fu.yml"
        self.targ_path = "/mnt/c/Users/zohai/Documents/Research/Neuro_ML/LVOs_Follow_Up_CTs/targets_fu.yml"
        self.itemData_init = pyConText_itemData_initializer(self.mod_path,self.targ_path)
        self.modifiers = self.itemData_init.get_itemData()[0]
        self.targets = self.itemData_init.get_itemData()[1]
        self.midline_shift_volume_pattern = re.compile('[0-9]+ mm')
        #Currently cm is flagged with -1, future work could involve processing cm values
#         self.midline_shift_cm_pattern = re.compile('[0-9]+\.*[0-9]* cm')
        
        #Outputs
        self.e_ms_se = []
        self.edema = []
        self.midline_shift = []
        self.sulcal_effacement = []
        
        for row in self.reports.itertuples():
            r1 = row.fu_CT_report
            r2 = row.fu_CT2_report
            r3 = row.fu_CT3_report
            row_reports = [r1,r2,r3]
            raw_e_ms_se_list = [self.extract_edema(r) for r in row_reports]
            e_ms_se_list = []
            for tup in raw_e_ms_se_list:
                if not None in tup:
                    e_ms_se_list.append(tup)
            
            self.e_ms_se.append(e_ms_se_list)
            
            #No report
            if e_ms_se_list == []:
                self.edema.append(None)
                self.midline_shift.append(None)
                self.sulcal_effacement.append(None)
            else:
                edema_list = [tup[0] for tup in e_ms_se_list]
                ms_list = [tup[1] for tup in e_ms_se_list]
                se_list = [tup[2] for tup in e_ms_se_list]
                self.edema.append(max(edema_list))
                self.midline_shift.append(max(ms_list))
                self.sulcal_effacement.append(max(se_list))
            
            
            
#             if str(report) != "nan":
#                 self.extract_edema(report)
#             else:
#                 self.edema.append(None)
#                 self.midline_shift.append(None)
#                 self.sulcal_effacement.append(None)
                
    def get_edema(self):
        return self.edema
    
    def get_midline_shift(self):
        return self.midline_shift
    
    def get_sulcal_effacement(self):
        return self.sulcal_effacement
    
    def get_raw(self):
        return self.e_ms_se
                
                    
                    
                    
    #Returns edema=0/1
    #Returns midline_shift=0/1
    #Returns sulcal_effacement=0/1
    
    def extract_edema(self,report):
        edema = None
        midline_shift = None
        sulcal_effacement = None
        #Report is nan
        if type(report) != str:
            return edema, midline_shift, sulcal_effacement

        sents = report.split(".")
        
        for sent in sents:
            ms = self.is_midline_shift(sent)
            ed = self.is_edema(sent)
            se = self.is_sulcal_effacement(sent)
          
            if ms is not None:
                if ms > 0:
                    search = self.midline_shift_volume_pattern.search(sent)
                    if search:
                        #Get '3' from 'there is 3 mm rightward midline shift'
                        midline_shift = int(search.group().split(" ")[0])
                    else:
#                         cm_search = self.midline_shift_cm_pattern.search(sent)
#                         if cm_search:
#                             print(cm_search.group().split(" ")[0])
#                             midline_shift = int(float(cm_search.group().split(" ")[0])*10)
                        #Debugging value for midline shift without volume estimate
                        midline_shift = -1
                else:
                    midline_shift = 0
            
            if ed is not None:
                if ed > 0:
                    edema = 1
                else:
                    edema = 0
            
            if se is not None:
                if se > 0:
                    sulcal_effacement = 1
                else:
                    sulcal_effacement = 0
                
            #print("Edema "+str(edema)+" midline_shift "+str(midline_shift))
            #Greedy algorithm, will only read sentences until it has answer for all 3
            if midline_shift is not None and edema is not None and sulcal_effacement is not None:
                break
        
        #Didn't find midline shift
        if midline_shift is None:
            midline_shift = 0
        #Didn't find edema
        if edema is None:
            edema = 0
        #Didn't find sulcal effacement
        if sulcal_effacement is None:
            sulcal_effacement = 0
            
        #In erroneous case of midline shift without edema
        if midline_shift > 1 and edema == 0:
            edema = 1
            
        return edema, midline_shift, sulcal_effacement
            
#         self.midline_shift.append(midline_shift)
#         self.edema.append(edema)
#         self.sulcal_effacement.append(sulcal_effacement)
        
        
        
        
    def is_midline_shift(self,sent):
        if "midline shift" in sent.lower():
            mtf = modifier_target_finder(sent,self.modifiers,self.targets)
            #0 for negation
            #1 for present in sent
            #2 for declared existence
            return mtf.confirm_existence("midline_shift","categoryString")
        return None
     
    def is_edema(self,sent):
        #Check for "edema" first
        if "edema" in sent.lower():
            mtf = modifier_target_finder(sent,self.modifiers,self.targets)
            #0 for negation
            #1 for present in sent
            #2 for declared existence
            return mtf.confirm_existence("edema","categoryString")
        return None
    
    def is_sulcal_effacement(self,sent):
        #Edema not found, check for 'effacement of sulci' or 'sulcal effacement' or loss of sulci
        if "sulc" in sent.lower():
            mtf = modifier_target_finder(sent,self.modifiers,self.targets)
            if mtf.find_mod_target_pair("effacement","categoryString","sulci","categoryString"):
                #0 for negation
                #1 for present in sent
                #2 for declared existence
                return mtf.confirm_existence("effacement","categoryString") 
            
        return None
    
            
        
        
                    
                
            

In [41]:
i=0
ee = edema_extractor(reports)
for row in reports.itertuples():
    r1 = row.fu_CT_report
    r2 = row.fu_CT2_report
    r3 = row.fu_CT3_report
    row_reports = [r1,r2,r3]
    ee_list = [ee.extract_edema(r) for r in row_reports]
    i+=1
    print(ee_list)
    if i == 4:
        break
new_ee = []
for tup in ee_list:
    if not None in tup:
        new_ee.append(tup)
new_ee
edema_list = [tup[2] for tup in new_ee]
edema_list
max(edema_list)

[(0, 0, 0), (None, None, None), (None, None, None)]
[(1, 4, 1), (None, None, None), (None, None, None)]
[(1, 3, 1), (1, 4, 1), (None, None, None)]
[(0, 0, 1), (0, 0, 1), (0, 0, 0)]


1

In [42]:
reports["fu_CT_report"][1]


'There is large right anterior circulation ischemic infarct with new 4 mm of leftward midline shift. Hyperdensity within the right sylvian fissure may represent vessel, blood products, or contrast. There is no evidence of intraparenchymal hemorrhage. There is diffuse sulcal effacement of the right cerebral hemisphere with diffuse loss of gray-white matter differentiation. The right lateral ventricle is effaced. The remainder of the ventricular system is nondilated. The basal cisterns are grossly patent.'

In [43]:
# ee_test = edema_extractor([reports["fu_CT_report"][2]])
# ee_test.edema


In [44]:
ee_test = edema_extractor(reports)
ee_test.edema

[0,
 1,
 1,
 0,
 None,
 0,
 0,
 None,
 None,
 1,
 None,
 1,
 1,
 1,
 None,
 None,
 0,
 1,
 0,
 1,
 0,
 None,
 None,
 0,
 None,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 None,
 None,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 None,
 1,
 None,
 0,
 1,
 0,
 None,
 0,
 None,
 None,
 1,
 0,
 0,
 1,
 1,
 None,
 None,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 None,
 None,
 1,
 None,
 0,
 1,
 0,
 1,
 None,
 0,
 1,
 1,
 None,
 None,
 None,
 None,
 None,
 1,
 0,
 None,
 None,
 1,
 0,
 0,
 None,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 None,
 None,
 None,
 None,
 1,
 None,
 1,
 1,
 None,
 None,
 1,
 0,
 1]

In [45]:
ee_test.midline_shift

[0,
 4,
 4,
 0,
 None,
 0,
 0,
 None,
 None,
 0,
 None,
 2,
 9,
 0,
 None,
 None,
 0,
 17,
 0,
 0,
 0,
 None,
 None,
 0,
 None,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 15,
 None,
 None,
 0,
 0,
 0,
 11,
 3,
 0,
 0,
 0,
 0,
 5,
 None,
 13,
 None,
 0,
 2,
 0,
 None,
 0,
 None,
 None,
 0,
 0,
 0,
 7,
 6,
 None,
 None,
 0,
 3,
 0,
 0,
 5,
 0,
 10,
 0,
 15,
 11,
 0,
 0,
 None,
 None,
 0,
 None,
 0,
 0,
 0,
 0,
 None,
 0,
 2,
 0,
 None,
 None,
 None,
 None,
 None,
 0,
 0,
 None,
 None,
 0,
 0,
 0,
 None,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 None,
 None,
 None,
 None,
 0,
 None,
 8,
 15,
 None,
 None,
 9,
 0,
 10]

In [46]:
ee = edema_extractor(reports)
ee.edema[:5]

[0, 1, 1, 0, None]

In [47]:
ee.midline_shift[:5]

[0, 4, 4, 0, None]

In [48]:
ee.sulcal_effacement[:5]

[0, 1, 1, 1, None]

In [49]:
edema_df = raw_df
edema_df["edema"] = ee.get_edema()
edema_df["midline shift (mm)"] = ee.get_midline_shift()
edema_df["sulcal effacement"] = ee.get_sulcal_effacement()
edema_df["ee_ms_se"] = ee.get_raw()
edema_df[:30]

,Unnamed: 0,Subject.label,MRN,GENISIS.CNDA,Accession.Number,first_name,last_name,dob,age.from.dob,age.radiology,...,DHC,Deterioration,Death,Osmotics,Comments,manual_mls,edema,midline shift (mm),sulcal effacement,ee_ms_se
0,1,BJH_001,42235384,NaN,53710639,MELODIE,SCHALK,1955-02-25,64.711841,64,...,0.0,0,0.0,0.0,0,NaN,0.0,0.0,0.0,"[(0, 0, 0)]"
1,2,BJH_002,2484331,NaN,53702325,SHARON,HOFFMAN,1946-05-23,73.470226,73,...,0.0,"CE, MLS",1.0,0.0,WOC,4.0,1.0,4.0,1.0,"[(1, 4, 1)]"
2,3,BJH_003,1052729,NaN,53682761,KATHLEEN,ERNST,1947-09-11,72.156057,72,...,0.0,"CE, MLS",0.0,0.0,0,4.0,1.0,4.0,1.0,"[(1, 3, 1), (1, 4, 1)]"
3,4,BJH_004,42914360,NaN,53681846,EDDIE,SIMS,1938-01-16,81.806982,81,...,0.0,0,0.0,0.0,0,NaN,0.0,0.0,1.0,"[(0, 0, 1), (0, 0, 1), (0, 0, 0)]"
4,5,BJH_005,5364336,NaN,53678983,MICHAEL,SANDERS,1955-06-30,64.355921,64,...,0.0,0,0.0,0.0,0,NaN,NaN,NaN,NaN,[]
5,6,BJH_006,43584850,NaN,53646960,CARL,POLAND,1988-01-18,31.789185,31,...,0.0,0,0.0,0.0,0,NaN,0.0,0.0,0.0,"[(0, 0, 0)]"
6,7,BJH_007,3225721,NaN,53645865,KENNETH,MOUNGO,1962-02-18,57.702943,57,...,0.0,0,1.0,0.0,"WOC - Hx of liver mets, not a candidate for pa...",NaN,0.0,0.0,0.0,"[(0, 0, 0)]"
7,8,BJH_008,5197806,NaN,53643358,VICTORIA,SPRAJCAR,1952-11-19,66.948665,67,...,0.0,0,0.0,0.0,0,NaN,NaN,NaN,NaN,[]
8,9,BJH_009,42637928,NaN,53604958,DANIEL,WALKER,1951-11-15,67.947981,68,...,0.0,0,0.0,0.0,0,NaN,NaN,NaN,NaN,[]
9,10,BJH_010,43578003,NaN,53557984,SHIRLEY,MARTY,1934-12-13,84.848734,84,...,0.0,CE,1.0,0.0,WOC,NaN,1.0,0.0,1.0,"[(1, 0, 1)]"


In [50]:
#Malignant edema derived from Deterioration ("ME" or "herniation"), DHC, Osmotics
edema_df["ME"] = 0
edema_df["DHC"].fillna(0,inplace=True)
edema_df["Osmotics"].fillna(0,inplace=True)
# edema_df["Deterioration"].fillna("",inplace=True)
edema_df["DHC"].astype("int")
edema_df["Osmotics"].astype("int")
# edema_df["Osmotics"].astype("str")

edema_df.loc[edema_df["DHC"]>0,"ME"] = 1
edema_df.loc[edema_df["Osmotics"]>0,"ME"] = 1
edema_df.loc[edema_df["Deterioration"].str.contains("ME",na=False),"ME"] = 1
edema_df.loc[edema_df["Deterioration"].str.contains("herniation",na=False),"ME"] = 1

edema_df[edema_df["ME"] > 0] 

,Unnamed: 0,Subject.label,MRN,GENISIS.CNDA,Accession.Number,first_name,last_name,dob,age.from.dob,age.radiology,...,Deterioration,Death,Osmotics,Comments,manual_mls,edema,midline shift (mm),sulcal effacement,ee_ms_se,ME
17,19,BJH_019,8166144,NaN,53415623,ROSALYN,HILL,1955-11-18,63.857632,63,...,"CE,MLS",1.0,1.0,100gm mannitol; progressed to BD,17.0,1.0,17.0,1.0,"[(1, 17, 1), (1, 9, 0)]",1
20,22,BJH_022,43547379,NaN,53292603,LEONARD,PAROTT,1956-03-03,63.518138,63,...,ME,0.0,0.0,0,NaN,0.0,0.0,0.0,"[(0, 0, 0), (0, 0, 0)]",1
31,33,BJH_033,43516493,NaN,53017602,CHARLES,HAMMER,1951-02-11,68.449008,68,...,"CE, ME",1.0,0.0,"WOC-refractory shock, AKI",NaN,1.0,0.0,0.0,"[(0, 0, 0), (1, 0, 0), (1, 0, 0)]",1
35,37,BJH_037,43504552,NaN,52903392,DAVON A,GRAHAM,1995-11-08,23.660507,23,...,"CE,MLS,herniation",1.0,1.0,"Mannitol, 23.4% NS, progressed to brain death",15.0,1.0,15.0,0.0,"[(1, 8, 0), (1, 15, 0), (1, 0, 0)]",1
39,41,BJH_041,42965893,NaN,52822771,AUGUSTA,TURNER,1990-04-24,29.160849,29,...,"CE,MLS,ME",0.0,0.0,0,-1.0,1.0,0.0,1.0,"[(1, -1, 1), (1, 0, 0), (1, 0, 0)]",1
41,43,BJH_043,41683494,Pending,52806155,BRITTANY,RANDALL,1985-11-13,33.598905,33,...,"CE,MLS,herniation",1.0,1.0,"Mannitol, WOC - gastric adenoca",11.0,1.0,11.0,1.0,"[(0, 0, 0), (1, 0, 1), (1, 11, 0)]",1
42,44,BJH_044,42544719,Pending,52779658,NORMA,TILSON,1933-02-07,86.351814,86,...,"MLS, ME",1.0,0.0,Comfort care-medical comorbidies,3.0,1.0,3.0,0.0,"[(0, 0, 0), (1, 3, 0), (0, 0, 0)]",1
47,49,BJH_049,5330984,NaN,52718578,BILLY,STARKS,1937-10-21,81.620808,81,...,"CE,MLS,ME",0.0,0.0,0,5.0,1.0,5.0,1.0,"[(1, 5, 1), (0, -1, 0)]",1
49,52,BJH_052,41760381,Pending,52656795,ROBERT,MILLER,1958-08-15,60.777550,60,...,"CE,MLS,herniation",0.0,0.0,0,13.0,1.0,13.0,0.0,"[(1, 8, 0), (1, 13, 0)]",1
52,55,BJH_055,3535873,NaN,52623273,BRIDGETT,STAGGERS,1980-12-19,38.417522,38,...,"CE, MLS, ME",0.0,0.0,0,2.0,1.0,2.0,0.0,"[(1, 2, 0), (0, 0, 0)]",1


In [51]:
edema_df["manual_edema"] = 0
edema_df.loc[edema_df["Deterioration"].str.contains("CE",na=False),"manual_edema"] = 1
edema_df["edema"].fillna(0,inplace=True)
edema_df["midline shift (mm)"].fillna(0,inplace=True)
edema_df["manual_mls"].fillna(0,inplace=True)
edema_df

,Unnamed: 0,Subject.label,MRN,GENISIS.CNDA,Accession.Number,first_name,last_name,dob,age.from.dob,age.radiology,...,Death,Osmotics,Comments,manual_mls,edema,midline shift (mm),sulcal effacement,ee_ms_se,ME,manual_edema
0,1,BJH_001,42235384,NaN,53710639,MELODIE,SCHALK,1955-02-25,64.711841,64,...,0.0,0.0,0,0.0,0.0,0.0,0.0,"[(0, 0, 0)]",0,0
1,2,BJH_002,2484331,NaN,53702325,SHARON,HOFFMAN,1946-05-23,73.470226,73,...,1.0,0.0,WOC,4.0,1.0,4.0,1.0,"[(1, 4, 1)]",0,1
2,3,BJH_003,1052729,NaN,53682761,KATHLEEN,ERNST,1947-09-11,72.156057,72,...,0.0,0.0,0,4.0,1.0,4.0,1.0,"[(1, 3, 1), (1, 4, 1)]",0,1
3,4,BJH_004,42914360,NaN,53681846,EDDIE,SIMS,1938-01-16,81.806982,81,...,0.0,0.0,0,0.0,0.0,0.0,1.0,"[(0, 0, 1), (0, 0, 1), (0, 0, 0)]",0,0
4,5,BJH_005,5364336,NaN,53678983,MICHAEL,SANDERS,1955-06-30,64.355921,64,...,0.0,0.0,0,0.0,0.0,0.0,NaN,[],0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,121,BJH_121,6121897,NaN,51506626,JOANN,THORNTON,1952-10-28,66.045175,66,...,0.0,0.0,0,0.0,0.0,0.0,NaN,[],0,0
118,122,BJH_122,40607632,NaN,51483615,GLADYS,CREMEENS,1928-08-29,90.195756,90,...,1.0,0.0,Hospice,0.0,0.0,0.0,NaN,[],0,0
119,123,BJH_123,42953119,NaN,51450558,STEPHEN,SWINEY,1952-01-03,66.839151,66,...,1.0,1.0,"Mannitol, WOC",9.0,1.0,9.0,0.0,"[(1, 5, 0), (1, 9, 0)]",1,1
120,124,BJH_124,42947670,NaN,51404081,EILEEN,COX,1932-03-20,86.606434,86,...,0.0,0.0,0,0.0,0.0,0.0,0.0,"[(0, 0, 0)]",0,0


In [52]:
from sklearn.metrics import classification_report
print(classification_report(edema_df["manual_edema"], edema_df["edema"]))

              precision    recall  f1-score   support

           0       0.96      0.97      0.97        77
           1       0.95      0.93      0.94        45

    accuracy                           0.96       122
   macro avg       0.96      0.95      0.96       122
weighted avg       0.96      0.96      0.96       122



In [53]:
mls = edema_df[edema_df["manual_mls"]>0]
print(classification_report(mls["manual_mls"], mls["midline shift (mm)"]))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      1.00      1.00         1
         2.0       1.00      1.00      1.00         2
         3.0       1.00      1.00      1.00         1
         4.0       1.00      0.75      0.86         4
         5.0       1.00      0.67      0.80         3
         6.0       0.00      0.00      0.00         1
         7.0       1.00      1.00      1.00         1
         8.0       1.00      1.00      1.00         1
         9.0       1.00      1.00      1.00         2
        10.0       1.00      1.00      1.00         2
        11.0       1.00      1.00      1.00         2
        13.0       1.00      1.00      1.00         1
        15.0       1.00      1.00      1.00         3
        17.0       1.00      1.00      1.00         1

    accuracy                           0.88        25
   macro avg       0.87      0.83      0.84        25
weighted avg       0.96   

/mnt/c/Users/zohai/Documents/Research/Neuro_ML/miniconda3/envs/neuroml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [54]:
#PRESENCE OF MLS
mls.loc[mls["manual_mls"]>0] = 1
mls.loc[mls["midline shift (mm)"]>0] = 1
print(classification_report(mls["manual_mls"], mls["midline shift (mm)"]))

              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00        25

    accuracy                           1.00        25
   macro avg       1.00      1.00      1.00        25
weighted avg       1.00      1.00      1.00        25



/mnt/c/Users/zohai/Documents/Research/Neuro_ML/miniconda3/envs/neuroml/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/mnt/c/Users/zohai/Documents/Research/Neuro_ML/miniconda3/envs/neuroml/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [55]:
output_filename = filename.split(".")[0]
output_filename += "_AUTOMATED_EDEMA_RESULTS.xlsx"
output_filename

'/mnt/c/Users/zohai/Documents/Research/Neuro_ML/LVOs_Follow_Up_CTs/BJH_2018_2019_master_v2_AUTOMATED_EDEMA_RESULTS.xlsx'

In [56]:
edema_df.to_excel(output_filename, index=False)